<a href="https://colab.research.google.com/github/abhignan05/excell-training/blob/main/Spam_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries from the NLTK toolkit
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize  # For tokenizing text into words and sentences

# Importing stopwords from NLTK to remove common words that add little value
from nltk.corpus import stopwords

# Downloading required NLTK datasets
nltk.download('punkt')  # Downloads tokenizer models for sentence and word tokenization
nltk.download('punkt_tab')  # Optional: Additional support for tokenization
nltk.download('stopwords')  # Downloads predefined stopword lists for various languages

# Downloading the dataset from Kaggle using Kaggle CLI
# Here, we download a dataset containing spam emails
!kaggle datasets download -d abdallahwagih/spam-emails  # Downloads the spam emails dataset
!unzip spam-emails.zip  # Extracts the downloaded dataset

# Importing pandas for data manipulation
import pandas as pd

# Loading the dataset into a pandas DataFrame
# The CSV file contains columns like 'Message' (email text) and 'Label' (spam/not spam indicator)
df = pd.read_csv("spam.csv")

# Step to clean the text data
# - Removing punctuation, special characters, and multiple spaces
# - Preparing the data for tokenization and further processing
import re  # Regular expressions module for text cleaning

cleaned = []  # List to store cleaned text
for text in df['Message']:  # Iterating over each message in the 'Message' column
    cleaned_text = re.sub(r'[^\w\s]', '', text)  # Removing punctuation and special characters
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Replacing multiple spaces with a single space
    cleaned_data = cleaned_text.strip()  # Removing leading and trailing spaces
    cleaned.append(cleaned_data)  # Appending the cleaned text to the list

# Tokenizing the cleaned text into words
# Each cleaned text is split into a list of individual words for further analysis
tokens = [word_tokenize(x) for x in cleaned]

# Removing stopwords from the tokenized words
# Stopwords are common words (e.g., "the", "is") that do not contribute much to analysis
stop = set(stopwords.words('english'))  # Fetching the list of English stopwords
stop_token = []  # List to store tokens after removing stopwords
for k in range(len(df['Message'])):  # Iterating through tokenized text
    p = [i for i in tokens[k] if i not in stop]  # Filtering out stopwords
    stop_token.append(p)  # Adding filtered tokens to the list

# Applying stemming to reduce words to their root form
# This helps group similar words (e.g., "running", "runner" -> "run")
from nltk.stem import PorterStemmer
ps = PorterStemmer()  # Initializing the Porter Stemmer
stemedata = []  # List to store stemmed data
for message in stop_token:  # Iterating over tokens after stopword removal
    st = [ps.stem(word) for word in message]  # Applying stemming to each word
    stemedata.append(st)  # Adding the stemmed tokens to the list

# Summary of steps:
# 1. Necessary libraries and NLTK datasets are imported/downloaded.
# 2. A spam email dataset is downloaded from Kaggle and loaded into a pandas DataFrame.
# 3. The 'Message' column is cleaned by removing punctuation, special characters, and extra spaces.
# 4. The cleaned text is tokenized into individual words.
# 5. Stopwords are removed from the tokenized words to reduce noise.
# 6. Stemming is applied to group similar words to their root form.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Dataset URL: https://www.kaggle.com/datasets/abdallahwagih/spam-emails
License(s): apache-2.0
  0% 0.00/207k [00:00<?, ?B/s]
100% 207k/207k [00:00<00:00, 70.3MB/s]
Archive:  spam-emails.zip
  inflating: spam.csv                


In [ ]:
#apply pos_tags on stop_token
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger_eng')


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
pos_tokens=[pos_tag(message) for message in stop_token]

In [ ]:
# now lets apply lemmatization on the data
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
lm=WordNetLemmatizer()

In [ ]:
lm.lemmatize('running',"v")

'run'

In [ ]:
# Function to convert nltk's POS tags to WordNet's POS tags
from nltk.corpus import wordnet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ  # Adjective
    elif tag.startswith('V'):
        return wordnet.VERB  # Verb
    elif tag.startswith('N'):
        return wordnet.NOUN  # Noun
    elif tag.startswith('R'):
        return wordnet.ADV  # Adverb
    else:
        return wordnet.NOUN  # Default to noun if

In [ ]:
get_wordnet_pos(pos_tokens[0][0][1])

'v'

In [ ]:
pos_tokens[0][0][1]

'VB'

In [ ]:
lm.lemmatize(pos_tokens[0][0][0],get_wordnet_pos(pos_tokens[0][0][1]))

'Go'

In [ ]:
lemmed_data=[]
for x in range(len(pos_tokens)):
  lem=[lm.lemmatize(pos_tokens[x][y][0],get_wordnet_pos(pos_tokens[x][y][1]))  for y in range(len(pos_tokens[x]))]
  lemmed_data.append(lem)

In [ ]:
from ctypes import c_void_p
#now we are applying count vectorization to convert
#text data to nummerical format using bagof words concept
#import countvectorization
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [ ]:
sen_stem=[' '.join(i) for i in stemedata]

In [ ]:
x_vec=cv.fit_transform(sen_stem).toarray()

In [ ]:
pd.Series(x_vec[0]).value_counts()

,count
0,8139
1,14


In [ ]:
pd.DataFrame([cv.get_feature_names_out(),x_vec[0]])

,0,1,2,3,4,5,6,7,8,9,...,8143,8144,8145,8146,8147,8148,8149,8150,8151,8152
0,008704050406,0089mi,0121,01223585236,01223585334,0125698789,02,020603,0207,02070836089,...,zed,zero,zhong,zindgi,zoe,zogtoriu,zoom,zouk,zyada,üll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_vec[0]

<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [ ]:
#importing Multimonal mode
from sklearn.naive_bayes import MultinomialNB


In [ ]:
mb=MultinomialNB()

In [ ]:
y=df['Category']

In [ ]:
mb.fit(x_vec,y)

MultinomialNB()

In [ ]:
mb.predict([x_vec[0]])

array(['ham'], dtype='<U4')